# Pondr: Fostering Connection through Good Conversation

We tend to chat all the time with friends new and olds, but often it feels like we’re just scratching at the surface, or defaulting to predictable, mundane conversations. Really good conversations — ones that introduce an element of vulnerability, spur a moment of fun, or create a deep sense of closeness — are few and far between. And when we have those types of conversations, we remember them.

Pondr is a game that turns strangers into friends, and friends into besties, by fostering connection and closeness through really good conversations. Using Cohere, Pondr generates question prompts on command that are uniquely tailored to the players’ setting. Whether you’re looking to deepen a connection with someone you’ve known forever, or you’re just hoping to become more familiar with a new friend, Pondr will help you drive the right sort of conversation.

You can build your own version of Pondr by following these simple steps: 

1. Generate potential conversation questions
2. Rank the generated questions
3. Put the generation and classification behind an interface

In this notebook we will walk through the first two steps.


### Setup
Install and import the tools we will need as well as initializing the Cohere model.

In [ ]:
# TODO: upgrade to "cohere>5"!pip install "cohere<5"

In [ ]:
import cohere
from cohere.responses.classify import Example
import pandas as pd

In [ ]:
co=cohere.Client('YOUR_API_KEY')

### 1) Generate Potential Conversation Questions
Generate a list of potential conversation questions and retain the first 10.

In [ ]:
#user_input is hardcoded for this example
user_input='I am meeting up with a coworker. We are meeting at a fancy restaurant. I wanna ask some interesting questions. These questions should be deep.'
prompt=user_input+'\nHere are 10 interesting questions to ask:\n1)'
response=co.generate(model='xlarge', prompt=prompt, max_tokens=200, temperature=5).generations[0].text
response

In [ ]:
def generation_to_df(generation):
    generation=response.split('\n')
    clean_questions=[]
    for i in range(10):
        curr_q=generation[i]
        clean_questions.append(curr_q[curr_q.find(')')+1:])
    clean_q_df=pd.DataFrame(clean_questions, columns=['questions'])
    return clean_q_df

In [ ]:
clean_q_df = generation_to_df(response)
pd.options.display.max_colwidth=150
clean_q_df

### 2) Classify Questions
Rank and sort the questions based on interestingness and specificity.

In [ ]:
# example labelled questions that act as a training samples for the classification
interestingness=[
    Example("What do you think is the hardest part of what I do for a living?", "Not Interesting"), 
    Example("What\'s the first thing you noticed about me?", "Interesting"), 
    Example("Do you think plants thrive or die in my care?", "Interesting"), 
    Example("Do I seem like more of a creative or analytical type?", "Interesting"), 
    Example("What subject do you think I thrived at in school?", "Not Interesting"), 
    Example("What\'s been your happiest memory this past year?", "Interesting"), 
    Example("What lesson took you the longest to un-learn?", "Not Interesting"), 
    Example("How can you become a better person?", "Not Interesting"), 
    Example("Do you think I intimidate others? Why or why not?", "Interesting"), 
    Example("What\'s the most embarrassing thing that happened to you on a date?", "Not Interesting"), 
    Example("How would you describe what you think my type is in three words?", "Interesting"), 
    Example("What do you think I\'m most likely to splurge on?", "Interesting"), 
    Example("As a child what do you think I wanted to be when I grow up?", "Interesting"), 
    Example("Do you think you are usually early, on time, or late to events?", "Not Interesting"), 
    Example("Do you think I was popular at school?", "Interesting"), 
    Example("What questions are you trying to answer most in your life right now?", "Not Interesting")]
specificity=[
    Example("What\'s the first thing you noticed about me?", "Specific"), 
    Example("Do you think plants thrive or die in my care?", "Specific"), 
    Example("Do I seem like more of a creative or analytical type?", "Not Specific"), 
    Example("How would you describe what you think my type is in three words?", "Not Specific"), 
    Example("What do you think I\'m most likely to splurge on?", "Specific"), 
    Example("What subject do you think I thrived at in school?", "Not Specific"), 
    Example("As a child what do you think I wanted to be when I grow up?", "Specific"), 
    Example("Do you think I was popular at school?", "Specific"), 
    Example("Do you think you\'re usually early, on time, or late to events?", "Specific"), 
    Example("Do you think I intimidate others? Why or why not?", "Specific"), 
    Example("What\'s been your happiest memory this past year?", "Not Specific"), 
    Example("What subject do you think I thrived at in school?", "Specific"), 
    Example("What\'s the biggest mistake that you think you needed to make to become who you are now?", "Specific"), 
    Example("Is there anything you\'ve done recently that you\'re incredibly proud of?", "Not Specific"), 
    Example("How are you and your siblings similar?", "Not Specific"), 
    Example("What\'s the worst pain you have ever been in that wasn\'t physical?", "Specific"), 
    Example("Has a stranger ever changed your life?", "Specific"), 
    Example("Do you think the image you have of yourself matches the image other people see you as?", "Specific"), 
    Example("What would your younger self not believe about your life today?", "Specific")]

In [ ]:
def add_attribute(df, attribute, name, target):

  response = co.classify(
    model='medium',
    inputs=list(df['questions']),
    examples=attribute)

  q_conf=[]
  for q in response.classifications:
    q_conf.append(q.labels[target].confidence)

  df[name]=q_conf

In [ ]:
add_attribute(clean_q_df, interestingness, 'interestingness', 'Interesting')
add_attribute(clean_q_df, specificity, 'specificity', 'Specific')
clean_q_df['average']= clean_q_df.iloc[:,1:].mean(axis=1)
clean_q_df.sort_values(by='average', ascending=False)